<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/autogen/gemini/gemini_custom_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -U -q autogen-agentchat autogen-ext[openai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 2.1 MB/s eta 0:00:00


In [5]:
%pip install -U -q autogen autogen_ext[openai] google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY

In [3]:
import os
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY
os.environ['GEMINI_API_KEY']=GEMINI_API_KEY

In [13]:
import autogen
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent
from autogen_ext.models import openai.O

In [7]:
genai.configure(api_key=GEMINI_API_KEY)

In [18]:
gemini_flash="gemini-2.0-flash"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"

In [19]:
def ask_gemini(prompt, model=gemini_flash):
    response = genai.GenerativeModel(model).generate_content(prompt)
    return response.text

In [40]:
# 3. Create Custom Agent
# 3. Create Custom Agent with Corrected Method Signature
class GeminiAgent(AssistantAgent):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

    def generate_reply(self, messages=None, sender=None, **kwargs):
        # Extract conversation history
        if messages is None:
            messages = self.chat_messages[sender]

        # Format for Gemini (simple concatenation)
        prompt = "\n".join([msg["content"] for msg in messages])

        # Get Gemini response
        response = ask_gemini(prompt)
        return response
# 4. Setup Agents
gemini_assistant = GeminiAgent(
    name="Gemini_Expert",
    system_message="""You are a helpful assistant. you are calling other assitant for work. When you are happy with response,
    end your response with 'TERMINATE'.""",
)

In [41]:
user_proxy = UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",  # Auto-reply
    max_consecutive_auto_reply=3,
    is_termination_msg=lambda x: x.get("content", "") == "" and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "coding"},
)


In [42]:
# 5. Start Conversation
user_proxy.initiate_chat(
    gemini_assistant,
    message="Explain quantum computing like I'm 10 years old. "
)

User_Proxy (to Gemini_Expert):

Explain quantum computing like I'm 10 years old. 

--------------------------------------------------------------------------------
Gemini_Expert (to User_Proxy):

Okay, imagine you have a light switch. It can be either ON or OFF, right? That's like a regular computer bit - it's either a 0 (OFF) or a 1 (ON).

Now, imagine a special light switch that can be ON, OFF, or BOTH at the SAME TIME!  It's like the light is kind of flickering! That's what a **quantum bit** (or **qubit**) is like!

Instead of just being 0 or 1, a qubit can be 0, 1, or a mix of both.  We call this mix "superposition". It's like the qubit is trying out both possibilities at once!

Think of it like a coin spinning in the air.  While it's spinning, it's not heads or tails yet, it's kind of both! Only when it lands do you know for sure if it's heads or tails.

Now, imagine you have a bunch of these special light switches (qubits) that are all connected to each other.  They can talk to e

ChatResult(chat_id=None, chat_history=[{'content': "Explain quantum computing like I'm 10 years old. ", 'role': 'assistant', 'name': 'User_Proxy'}, {'content': 'Okay, imagine you have a light switch. It can be either ON or OFF, right? That\'s like a regular computer bit - it\'s either a 0 (OFF) or a 1 (ON).\n\nNow, imagine a special light switch that can be ON, OFF, or BOTH at the SAME TIME!  It\'s like the light is kind of flickering! That\'s what a **quantum bit** (or **qubit**) is like!\n\nInstead of just being 0 or 1, a qubit can be 0, 1, or a mix of both.  We call this mix "superposition". It\'s like the qubit is trying out both possibilities at once!\n\nThink of it like a coin spinning in the air.  While it\'s spinning, it\'s not heads or tails yet, it\'s kind of both! Only when it lands do you know for sure if it\'s heads or tails.\n\nNow, imagine you have a bunch of these special light switches (qubits) that are all connected to each other.  They can talk to each other and infl